In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

try: # When on google Colab, let's clone the notebook so we download the cache.
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    !pip install openai~=0.28.1
    # !pip install -e $repo_path

import dspy

fatal: cannot change to 'dspy': No such file or directory
Cloning into 'dspy'...
remote: Enumerating objects: 32902, done.
remote: Counting objects: 100% (699/699), done.
remote: Compressing objects: 100% (306/306), done.
remote: Total 32902 (delta 443), reused 590 (delta 379), pack-reused 32203
Receiving objects: 100% (32902/32902), 29.02 MiB | 6.91 MiB/s, done.
Resolving deltas: 100% (15718/15718), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.1/273.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [ ]:
import dspy
import openai
from dspy import OpenAI

openai.api_base = "https://api.gpts.vin/v1"  # use the IP or hostname of your instance
openai.api_key = "sk-FhKeNgB7ZgqvpZqsF357D67eAa8c42339b02Cf57C56a7d19"

lm = OpenAI(api_base = openai.api_base,
             api_key=openai.api_key,
             model='gpt-3.5-turbo',
             )
dspy.settings.configure(lm=lm)


In [2]:
# Create a few question–answer pairs for our task

train = [("sarah palin is awesome. | yes she is", "sarah palin", "positive"),
      ("russell brand and katy perry suit each other. they 're both weird.","katy perry", "negative"),
      ("lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when ...", "lindsay lohan", 'neutral'),
      ('and he has a vision too ! mirzan mahatir is looking like steve jobs right now.', 'steve jobs','neutral'),
      ("charlie sheen 's porn whore dumps him via sms while he 's on stage in florida . still '' winning '' charles ? lol . when keeping it real goes wrong !!", "charlie sheen","neutral"),
      ("i love google ! as a whole with all its services","google","positive")
        ]

train = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence') for sentence, aspect, sentiment in train]

In [3]:
dev = [("my 3-year-old was amazed yesterday to find that ' real ' 10 pin bowling is nothing like it is on the wii ...","wii","neutral"),
    ("listening 2 '' 3 '' by ms britney spears cant wait 4 the video on oct . 19 you guys need 2 love her new single also buy her single collection nov . 24 !","britney spears","positive"),
    ("ff markieai likes animals . hates spiders . hates charlie sheen . likes cookies .","charlie sheen","negative"),
    ("espn soccer net news ardiles backs maradona for world cup glory","maradona","positive"),
    ("-lsb- happy birthday git * git ! -rsb- ily ! ... some reason im very tired ! but today was really fun ! :-rrb- ugh my ipod died !","ipod","neutral"),
    ("hilary swank as amelia earhart is actually pretty great casting .","hilary swank","positive"),
    ("video : serena williams gets x-rated for new tennis game ad ! .","serena williams","negative"),
    ("lmao i know !! i dnt like harry potter he 's so english !! hehehe its raining here too smdh","harry potter","negative"),
    ("listening to britney spears ' 3 ! awesome song . sounds like an orgasm !","britney spears","positive")]
dev = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence') for sentence, aspect, sentiment in dev]

In [ ]:
# class BasicQA(dspy.Signature):
    #question = dspy.InputField()
    #answer = dspy.OutputField(desc="sentiment label often between positive, neutral and negative")

In [ ]:
# 对一个句子进行情感极性判断
import dspy

# Define the DSPy implementation of sentiment polarity analysis
class SentimentPolarityAnalysis(dspy.Signature):
    """Analyze the sentiment polarity of a given sentence."""

    sentence = dspy.InputField(desc="The sentence to analyze.")
    sentiment = dspy.OutputField(desc="The predicted sentiment polarity (positive, neutral, or negative).")

class SentimentAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze_sentiment = dspy.ChainOfThought(SentimentPolarityAnalysis)

    def forward(self, sentence):
        prediction = self.analyze_sentiment(sentence=sentence)
        return dspy.Prediction(sentiment=prediction.sentiment)

    def analyze_sentiment(self, sentence):
        prompt = f"请分析以下句子的情感极性（积极，中立，消极）：\n句子: {sentence}\n情感极性: "
        response = lm.complete(prompt=prompt)
        return response.choices[0].message.content

# Example usage
analyzer = SentimentAnalyzer()
sentence = "这是一个美好的一天。"
result = analyzer(sentence)
print(f"句子: {sentence}\n情感极性: {result.sentiment}")

句子: 这是一个美好的一天。
情感极性: Positive


In [ ]:
# 对一个句子中的aspect进行情感极性判断
import dspy

# Define the DSPy implementation of sentiment polarity analysis with an aspect
class SentimentPolarityAnalysis(dspy.Signature):
    """Analyze the sentiment polarity of a given sentence and its aspect."""

    sentence = dspy.InputField(desc="The sentence to analyze.")
    aspect = dspy.InputField(desc="The aspect within the sentence to analyze.")
    sentiment = dspy.OutputField(desc="The predicted sentiment polarity (positive, neutral, or negative).")

class SentimentAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze_sentiment = dspy.ChainOfThought(SentimentPolarityAnalysis)

    def forward(self, sentence, aspect):
        prediction = self.analyze_sentiment(sentence=sentence, aspect=aspect)
        return dspy.Prediction(sentiment=prediction.sentiment)

    def analyze_sentiment(self, sentence, aspect):
        prompt = f"请分析以下句子中 '{aspect}' 的情感极性（积极，中立，消极）：\n句子: {sentence}\n情感极性: "
        response = lm.complete(prompt=prompt)
        return response.choices[0].message.content

# Example usage
analyzer = SentimentAnalyzer()
example = {'sentence': 'sarah palin is awesome.', 'aspect': 'sarah palin'}
result = analyzer(example['sentence'], example['aspect'])
print(f"Sentence: {example['sentence']}\nAspect: {example['aspect']}\nSentiment: {result.sentiment}")

Sentence: sarah palin is awesome.
Aspect: sarah palin
Sentiment: positive


In [4]:
# 对一个句子中的aspect进行情感极性判断
import dspy
import openai
from dspy import OpenAI

openai.api_base = "https://api.gpts.vin/v1"
openai.api_key = "sk-FhKeNgB7ZgqvpZqsF357D67Eaa8c42339b02Cf57C56a7d19"

lm = OpenAI(api_base=openai.api_base,
             api_key=openai.api_key,
             model='gpt-3.5-turbo')

dspy.settings.configure(lm=lm)

# Define the DSPy implementation of sentiment polarity analysis with an aspect
class SentimentPolarityAnalysis(dspy.Signature):
    """Analyze the sentiment polarity of a given sentence and its aspect."""

    context = dspy.InputField(desc="Introduction of the aspect within the sentence.")
    sentence = dspy.InputField(desc="The sentence to analyze.")
    aspect = dspy.InputField(desc="The aspect within the sentence to analyze.")
    sentiment = dspy.OutputField(desc="The predicted sentiment polarity (positive, neutral, or negative).")

class SentimentAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze_sentiment = dspy.ChainOfThought(SentimentPolarityAnalysis)

    def forward(self, sentence, aspect):
        # Use GPT to create context introduction for the aspect
        context = self.get_aspect_context(aspect)

        prediction = self.analyze_sentiment(context=context, sentence=sentence, aspect=aspect)
        return dspy.Prediction(sentiment=prediction.sentiment)

    def get_aspect_context(self, aspect):
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"介绍一下'{aspect}'的背景。"}
            ],
            max_tokens=50,
            n=1,
            stop=None
        )
        context = response.choices[0].message['content'].strip()
        return context

    def analyze_sentiment(self, context, sentence, aspect):
        prompt = (
            f"请分析以下句子中 '{aspect}' 的情感极性（积极，中立，消极）：\n"
            f"背景: {context}\n"
            f"句子: {sentence}\n"
            f"情感极性: "
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,
            n=1,
            stop=None
        )
        return response.choices[0].message['content'].strip()

# Example usage
analyzer = SentimentAnalyzer()
example = {'sentence': 'sarah palin is awesome.', 'aspect': 'sarah palin', 'sentiment': 'positive'}
result = analyzer.forward(example['sentence'], example['aspect'])
print(f"Sentence: {example['sentence']}\nAspect: {example['aspect']}\nSentiment: {result.sentiment}")

Sentence: sarah palin is awesome.
Aspect: sarah palin
Sentiment: The sentiment polarity is positive.


In [ ]:
# Evluate sentiment analysis
from dspy.evaluate import Evaluate, normalize_text

def gold_sentiment(example, pred, trace=None):
    # Normalize the gold standard sentiment and predicted sentiment
    gold_sent_label = normalize_text(example['sentiment'])
    generated_sent_label = normalize_text(pred.sentiment)

    # Check if the predicted sentiment matches the gold standard sentiment
    return gold_sent_label.lower() == generated_sent_label.lower()

# 大小写统一; 句子中出现positive就算positive
# 备份一份做基础实验
# 类比prompt
# 了解优化器的作用 MIPRO OPRO
# 每个测试存储样例推理的过程，为什么GPT是这么推理的

Define a **Dataset**

In [5]:
train = [("sarah palin is awesome. | yes she is", "sarah palin", "positive"),
      ("russell brand and katy perry suit each other. they 're both weird.","katy perry", "negative"),
      ("lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when ...", "lindsay lohan", 'neutral'),
      ('and he has a vision too ! mirzan mahatir is looking like steve jobs right now.', 'steve jobs','neutral'),
      ("i love google ! as a whole with all its services","google","positive")
      ]

# 修改train数据集，使其包含输入`aspect`
train = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in train]

# Check each entry in the dev dataset for errors
dev = [("my 3-year-old was amazed yesterday to find that ' real ' 10 pin bowling is nothing like it is on the wii ...", "wii", "neutral"),
    ("listening 2 '' 3 '' by ms britney spears cant wait 4 the video on oct . 19 you guys need 2 love her new single also buy her single collection nov . 24 !", "britney spears", "positive"),
    ("ff markieai likes animals . hates spiders . hates charlie sheen . likes cookies .", "charlie sheen", "negative"),
    ("espn soccer net news ardiles backs maradona for world cup glory", "maradona", "positive"),
    ("-lsb- happy birthday git * git ! -rsb- ily ! ... some reason im very tired ! but today was really fun ! :-rrb- ugh my ipod died !", "ipod", "neutral"),
    ("hilary swank as amelia earhart is actually pretty great casting .", "hilary swank", "positive"),
    ("video : serena williams gets x-rated for new tennis game ad ! .", "serena williams", "negative"),
    ("lmao i know !! i dnt like harry potter he 's so english !! hehehe its raining here too smdh", "harry potter", "negative"),]
# 修改dev数据集，使其包含输入`aspect`
dev = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in dev]

Define the **Metric**

In [6]:
def sentiment_classification_metric(gold, pred, trace=None):
    return gold.sentiment.lower() == pred.sentiment.lower()

Create the **DSPy Program**

In [7]:
import dspy

openai.api_base = "https://api.gpts.vin/v1"
openai.api_key = "sk-FhKeNgB7ZgqvpZqsF357D67Eaa8c42339b02Cf57C56a7d19"

lm = OpenAI(api_base=openai.api_base,
             api_key=openai.api_key,
             model='gpt-3.5-turbo')

dspy.settings.configure(lm=lm)
# Define the DSPy implementation of sentiment polarity analysis with an aspect
class SentimentPolarityAnalysis(dspy.Signature):
    """Analyze the sentiment polarity of a given sentence and its aspect."""

    sentence = dspy.InputField(desc="The sentence to analyze.")
    aspect = dspy.InputField(desc="The aspect within the sentence to analyze.")
    sentiment = dspy.OutputField(desc="The predicted sentiment polarity (positive, neutral, or negative).")

class SentimentAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze_sentiment = dspy.ChainOfThought(SentimentPolarityAnalysis)

    def forward(self, sentence, aspect):
        prediction = self.analyze_sentiment(sentence=sentence, aspect=aspect)
        return dspy.Prediction(sentiment=prediction.sentiment)

    def analyze_sentiment(self, sentence, aspect):
        prompt = f"Please analyze the emotional polarity (positive, neutral, negative) of '{aspect}' in the following sentences:\n "
        response = lm.complete(prompt=prompt)
        return response.choices[0].message.content

Design a introduction context **Program**

In [8]:
import dspy
import openai
from dspy import OpenAI

openai.api_base = "https://api.gpts.vin/v1"
openai.api_key = "sk-FhKeNgB7ZgqvpZqsF357D67Eaa8c42339b02Cf57C56a7d19"

lm = OpenAI(api_base=openai.api_base,
             api_key=openai.api_key,
             model='gpt-3.5-turbo')

dspy.settings.configure(lm=lm)

# Define the DSPy implementation of sentiment polarity analysis with an aspect
class IntroSentimentPolarityAnalysis(dspy.Signature):
    """Analyze the sentiment polarity of a given sentence and its aspect."""
    sentence = dspy.InputField(desc="The sentence to analyze.")
    aspect = dspy.InputField(desc="The aspect within the sentence to analyze.")
    context = dspy.InputField(desc="Introduction of the aspect within the sentence.")
    sentiment = dspy.OutputField(desc="The predicted sentiment polarity (positive, neutral, or negative).")

class IntroSentimentAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_context = dspy.ChainOfThought(IntroSentimentPolarityAnalysis)
        self.analyze_sentiment = dspy.ChainOfThought(IntroSentimentPolarityAnalysis)

    def forward(self, sentence, aspect):
        # Use GPT to create context introduction for the aspect
        context = self.get_aspect_context(aspect)

        prediction = self.analyze_sentiment(context=context, sentence=sentence, aspect=aspect)
        return dspy.Prediction(context=context, sentiment=prediction.sentiment)

    def get_aspect_context(self, aspect):
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Please breifly introduce '{aspect}'."}
            ],
            max_tokens=50,
            n=1,
            stop=None
        )

        context = response.choices[0].message['content'].strip()
        return context

    def analyze_sentiment(self, sentence, aspect, context):
        prompt = (
            f"Please analyze the emotional polarity (positive, neutral, negative) of '{aspect}' in the following sentences:\n"
            f"The introduction of '{aspect}' is {context}\n"
            f"sentence: {sentence}\n"
            f"The predicted sentiment polarity: "
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,
            n=1,
            stop=None
        )

        context_sentiment = response.choices[0].message['content'].strip()
        return context_sentiment

**Compile** the Program

In [9]:
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
teleprompter = BootstrapFewShot(metric=sentiment_classification_metric, **config)

# Compile the program
optimized_classifier = teleprompter.compile(IntroSentimentAnalyzer(), trainset=train)

dspy/cache/compiler


100%|██████████| 5/5 [01:22<00:00, 16.58s/it]


**Evaluate** the Program

In [10]:
from dspy.evaluate import Evaluate

# Set up the evaluator
evaluate = Evaluate(devset=dev, metric=sentiment_classification_metric, num_threads=4, display_progress=True, display_table=0)

# Evaluate the optimized classifier
evaluate(optimized_classifier)

Average Metric: 2 / 8  (25.0): 100%|██████████| 8/8 [01:03<00:00,  7.91s/it]


25.0

In [11]:
lm.inspect_history(n=1)




Analyze the sentiment polarity of a given sentence and its aspect.

---

Sentence: lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when ...
Aspect: lindsay lohan
Sentiment: neutral

Sentence: russell brand and katy perry suit each other. they 're both weird.
Aspect: katy perry
Sentiment: negative

---

Follow the following format.

Sentence: The sentence to analyze.

Aspect: The aspect within the sentence to analyze.

Context: Introduction of the aspect within the sentence.

Reasoning: Let's think step by step in order to ${produce the sentiment}. We ...

Sentiment: The predicted sentiment polarity (positive, neutral, or negative).

---

Sentence: sarah palin is awesome. | yes she is

Aspect: sarah palin

Context: Sarah Palin is an American politician, commentator, and author who rose to prominence as the Governor of Alaska from 2006 to 2009. She was the first female vice presidential candidate on the Republican ticke

'\n\n\nAnalyze the sentiment polarity of a given sentence and its aspect.\n\n---\n\nSentence: lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you \'ve hit rock bottom when ...\nAspect: lindsay lohan\nSentiment: neutral\n\nSentence: russell brand and katy perry suit each other. they \'re both weird.\nAspect: katy perry\nSentiment: negative\n\n---\n\nFollow the following format.\n\nSentence: The sentence to analyze.\n\nAspect: The aspect within the sentence to analyze.\n\nContext: Introduction of the aspect within the sentence.\n\nReasoning: Let\'s think step by step in order to ${produce the sentiment}. We ...\n\nSentiment: The predicted sentiment polarity (positive, neutral, or negative).\n\n---\n\nSentence: sarah palin is awesome. | yes she is\n\nAspect: sarah palin\n\nContext: Sarah Palin is an American politician, commentator, and author who rose to prominence as the Governor of Alaska from 2006 to 2009. She was the first female vice preside